# 2層ニューラルネットのクラスの実装

In [2]:
import sys, os
import numpy as np

def sigmoid(x):
    """
    シグモイド関数
    """
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # オーバーフロー対策でMax値を引く
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        # random.randnは正規分布による 10*2の行列をランダムに作成している
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        
    def predict(self, x):
        w1, w2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        
        return y
        
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_w, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_w, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])

        return grads

In [3]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
# 推論処理(フォワード)
x = np.random.rand(100, 784)
y = net.predict(x)

# 勾配計算
t = np.random.rand(100, 10)
grads = net.numerical_gradient(x, t)

In [5]:
y

array([[0.00099413, 0.00101738, 0.0010375 , 0.00102731, 0.00103791,
        0.00102932, 0.00090331, 0.00086964, 0.00101731, 0.00105196],
       [0.00099857, 0.00101992, 0.00103805, 0.00102962, 0.00103579,
        0.00102947, 0.00090063, 0.00086437, 0.00101973, 0.00105291],
       [0.00099615, 0.00102348, 0.00103698, 0.00102938, 0.00104147,
        0.00103598, 0.00090132, 0.00086763, 0.00101873, 0.00104789],
       [0.00099763, 0.0010154 , 0.00104198, 0.00103304, 0.00103591,
        0.00103096, 0.00090012, 0.00086723, 0.00102208, 0.00105597],
       [0.00099786, 0.00102413, 0.00104028, 0.00102804, 0.0010377 ,
        0.00103257, 0.00090083, 0.00086736, 0.00102113, 0.00105328],
       [0.00099753, 0.00102181, 0.00104011, 0.00102987, 0.00104002,
        0.00103343, 0.0008988 , 0.00086559, 0.00102187, 0.00104957],
       [0.00099388, 0.00102074, 0.00104122, 0.00103235, 0.00103741,
        0.00103209, 0.00089876, 0.00086898, 0.00101654, 0.00105595],
       [0.00099653, 0.00102087, 0.0010390